In [33]:
import pandas as pd
import numpy as np

In [35]:
biden_data = pd.read_csv('raw_data/2020_tweets/hashtag_donaldtrump.csv', lineterminator='\n')
trump_data = pd.read_csv('raw_data/2020_tweets/hashtag_joebiden.csv', lineterminator='\n')
biden_data['candidate'] = 'biden'
trump_data['candidate'] = 'trump'
data = pd.concat([trump_data, biden_data])

labeled_tweets = pd.read_csv('results/labeled_tweets_georgetown_model.csv')

labeled_tweets = labeled_tweets.merge(data[['tweet_id', 'user_id']], on='tweet_id', how='left')

FileNotFoundError: [Errno 2] No such file or directory: 'raw_data/2020_tweets/hashtag_donaldtrump.csv'

In [36]:
labeled_tweets = pd.read_csv('results/labeled_tweets_twitter_roberta.csv')

In [37]:
election_results = pd.read_csv('results/state_election_results.csv')

labeled_tweets['state_lower'] = labeled_tweets['state'].str.lower()
election_results['state_lower'] = election_results['state'].str.lower()

labeled_tweets = labeled_tweets[labeled_tweets['state_lower'].isin(election_results['state_lower'])]

labeled_tweets = labeled_tweets.drop(columns=['state_lower'])
election_results = election_results.drop(columns=['state_lower'])

In [38]:
# From https://ballotpedia.org/Presidential_battleground_states,_2020
swing_states = ['Arizona', 'Florida', 'Georgia', 'Iowa', 'Michigan',
                'Minnesota', 'Nevada', 'New Hampshire', 'North Carolina',
                'Ohio', 'Pennsylvania', 'Texas', 'Wisconsin']
labeled_tweets[labeled_tweets['state'].isin(swing_states)]['state'].value_counts()

state
Texas             14225
Florida           12025
Pennsylvania       6140
Georgia            4697
Ohio               4157
Nevada             3467
Arizona            3337
North Carolina     2329
Michigan           2088
Minnesota          1574
Wisconsin           991
Iowa                292
New Hampshire        50
Name: count, dtype: int64

In [ ]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None, 'display.max_colwidth', None):
    display(labeled_tweets.sample(50, replace=False, random_state=42)[['tweet', 'label']])

In [ ]:
multiple_states = labeled_tweets.groupby('user_id')['state'].apply(lambda x: list(x.unique()))
users_with_multiple_states = multiple_states[multiple_states.apply(len) > 1]

if not users_with_multiple_states.empty:
    print(f'There are {len(users_with_multiple_states)} users with multiple states')
    for user_id, states in users_with_multiple_states.items():
        print(f"User ID: {user_id}, States: {states}")
else:
    print("No users have multiple states.")


In [ ]:
# Using population vs twitter users from https://www.pewresearch.org/internet/2019/04/24/sizing-up-twitter-users/

stance_weights = pd.DataFrame({
    'forced_stance': [-1, 1, 0],
    'user_proportion': [0.6, 0.35, 0.05], 
    'population_proportion': [0.52, 0.43, 0.05]
    })

stance_weights['weight'] = stance_weights['population_proportion'] / stance_weights['user_proportion']
stance_weights = stance_weights[['forced_stance', 'weight']]

In [ ]:
# independents alliance https://www.pewresearch.org/politics/2019/03/14/political-independents-who-they-are-what-they-think/
proportions = np.array([17, 7, 13])
proportions = np.divide(proportions, proportions.sum())

def generate_forced_stance(avg_stance):
    if avg_stance < 0:
        return -1
    elif avg_stance > 0:
        return 1
    else:
        return np.random.choice([-1, 0, 1], p=proportions)

In [ ]:
users_stances = labeled_tweets.groupby('user_id', as_index=False).agg(
    avg_stance=('stance', 'mean'),
    state=('state', lambda x: x.mode().iloc[0])
)

users_stances['forced_stance'] = users_stances['avg_stance'].apply(generate_forced_stance)
users_stances = users_stances.merge(stance_weights, on='forced_stance')

In [ ]:
def weighted_avg(data, val, weight):
    return (data[val] * data[weight]).sum() / data[weight].sum()


weighted_avg_forced = (
    users_stances
    .groupby('state')
    .apply(lambda x: weighted_avg(x, 'forced_stance', 'weight'))
    .reset_index(name='weighted_avg_forced_stance')
)

In [ ]:
state_scores = users_stances.groupby('state', as_index=False).agg(
    average_avg_stance=('avg_stance', 'mean'),
    average_forced_stance=('forced_stance', 'mean')
)

user_counts = users_stances.groupby('state')['user_id'].nunique()
state_scores = state_scores.merge(user_counts.rename('user_count'), on='state')
state_scores = state_scores.merge(weighted_avg_forced, on='state')

state_scores[state_scores['state'].isin(swing_states)]

In [ ]:
reddit_data = pd.read_pickle('raw_data/factoid_reddit/reddit_corpus_unbalanced_filtered.gzip', compression='gzip')

columns_to_keep = [
    'pb_factor', 'user_id'
]

reddit_data = reddit_data[columns_to_keep]
reddit_data = pd.DataFrame({
    "user_id": reddit_data["user_id"],
    "stance": reddit_data["pb_factor"].apply(lambda x: -1 if x < -0.5 else (1 if x > 0.5 else 0))
})
reddit_data = reddit_data.reset_index(drop=True)


reddit_user_stances = pd.DataFrame({
    'user_id' : reddit_data.groupby('user_id')['stance'].mean().index,
    'stance' : reddit_data.groupby('user_id')['stance'].mean().values
})

In [ ]:
print(f"The average national average stance is {users_stances['avg_stance'].mean()}")
print(f"The average national forced stance is {users_stances['forced_stance'].mean()}")
#print(f"The average stance from reddit data is {reddit_user_stances['stance'].mean()}")

In [ ]:
state_scores.to_csv('results/state_scores_roberta_model.csv')